In [8]:
%gui qt

In [9]:
import numpy as np
import tifffile
import skimage
from tifffile import imread
import napari
import pandas as pd
from matplotlib.backends.backend_qt5agg import FigureCanvas
from matplotlib.figure import Figure
from pathlib import Path
from magicgui import event_loop, magicgui
from PyQt5.QtWidgets import QMessageBox
import sys
from PyQt5.QtWidgets import QApplication, QPushButton
from PyQt5.QtCore import Slot
import enum
import os

In [10]:
viewer=napari.Viewer()

In [11]:
@magicgui(call_button='Get Masks', layout='vertical')
def maskpicker(wholecount_mask_name=Path(),tablename=Path()):
    global img,mask_name,df_core_cellid,celltypes
    tname = str(tablename)

    if wholecount_mask_name.suffix == '.tif' or wholecount_mask_name.suffix == '.tiff':
        img = tifffile.imread(wholecount_mask_name)
        mask_name = Path(wholecount_mask_name).stem

        if tname == '.':
            viewer.add_labels(img, name=mask_name, num_colors=2)


        elif tablename.suffix == '.csv':
            df_data = pd.read_csv(tname)
            unique=np.unique(img)
            cellids=np.delete(unique,np.where(unique == 0)[0])
            df_indexed = df_data.set_index('CellId')
            cut_sample=df_data[df_data["CellId"].isin(cellids)]
                
            #if 'ID' in cut_sample.columns and 'GlobalCellType' in cut_sample.columns:
            if 'CellId' in cut_sample.columns:
                df_core_cellid = cut_sample.set_index('CellId')
                #celltypes = df_core_cellid["GlobalCellType"].unique()
                                    #color = {1: 'green', 2: 'orange', 3: 'purple', 4: 'papayawhip', 5: 'sienna'}

                rows = img.shape[0]
                cols = img.shape[1]
                c = 1
                global colorlist
                colorlist = []

                #for l in range(len(celltypes)):
                CelltypeMask = np.zeros(shape=(rows,cols),dtype=np.uint16)

                for i in range(0,rows):
                    for j in range(0,cols):
                        c_id = img[i,j]
                        if c_id in df_core_cellid.index:
                        #and df_core_cellid['GlobalCellType'][c_id] == celltypes[l]:
                            CelltypeMask[i,j] = c  
                viewer.add_labels(CelltypeMask, name='_Mask', visible = False)
                #viewer.add_labels(CelltypeMask, name=celltypes[l] + '_Mask', visible = False)
                #colorlist.append(viewer.active_layer.get_color(c))
                #c = c + 1

    return img, df_core_cellid, mask_name

#use this with napari3.7 (not sure where they changed it)
#gui = maskpicker.Gui(show=True)
#viewer.window.add_dock_widget(gui, area='right')

#use this with napari 4.9 and above
viewer.window.add_dock_widget(maskpicker)

In [12]:
@magicgui(call_button='Get lost_found', layout='vertical')
def lostandfound(wholecount_mask_name=Path(),tablename=Path()):
    global img,mask_name,df_core_cellid,celltypes
    tname = str(tablename)

    if wholecount_mask_name.suffix == '.tif' or wholecount_mask_name.suffix == '.tiff':
        img = tifffile.imread(wholecount_mask_name)
        mask_name = Path(wholecount_mask_name).stem

        if tname == '.':
            viewer.add_labels(img, name=mask_name, num_colors=2)


        elif tablename.suffix == '.csv':
            df_data = pd.read_csv(tname)
            if 'CellId' in df_data.columns and 'lost' in df_data.columns:
                viewer.add_labels(img, name=mask_name, visible = False, num_colors=2)
                df_core_cellid = df_data.set_index('CellId')
                celltypes = df_core_cellid["lost"].unique()
                #color = {1: 'green', 2: 'orange', 3: 'purple', 4: 'papayawhip', 5: 'sienna'}

                rows = img.shape[0]
                cols = img.shape[1]
                c = 1

                for l in range(len(celltypes)):
                    CelltypeMask = np.zeros(shape=(rows,cols),dtype=np.uint16)

                    for i in range(0,rows):
                        for j in range(0,cols):
                            id = img[i,j]
                            if id in df_core_cellid.index and df_core_cellid['lost'][id] == celltypes[l]:
                                CelltypeMask[i,j] = c
                    if celltypes[l] == True:
                        cname = 'Lost cells'
                    elif celltypes[l] == False:
                        cname = 'Found cells'
                    else:
                        cname = 'N/A'
                    
                    viewer.add_labels(CelltypeMask, name=cname, visible = False)
                    c = c + 1
                    
    
    return  img, df_core_cellid, mask_name, celltypes
#use this with napari3.7 (not sure where they changed it)
#gui = maskpicker.Gui(show=True)
#viewer.window.add_dock_widget(gui, area='right')

#use this with napari 4.9 and above
viewer.window.add_dock_widget(lostandfound)

In [13]:
 @magicgui(call_button='Get Channels', layout='vertical') #to choose directory add:  core_name={'mode' : 'd'}
def corepicker(core_name=Path(),channel_name_table=Path()):
    global df_name 
    df_name = str(channel_name_table)

    if core_name.suffix == '.tif' or core_name.suffix == '.tiff':
        if df_name == '.':
            with tifffile.TiffFile(core_name) as tif:
                for page in tif.pages:
                    for tag in page.tags:
                        tag_name, tag_value = tag.name, tag.value

                    image = page.asarray()
                    viewer.add_image(image, visible=False, contrast_limits=(0,65535), blending='additive')

        elif channel_name_table.suffix == '.xlsx' :
            df = pd.read_excel(df_name)

            if 'Channel_number' in df.columns and 'Channel_name' in df.columns:
                df_indexed = df.set_index('Channel_number')
                i = 1
                    
                if 'Cycle' in df.columns and 'Fluorochrome' in df.columns:
                    df_indexed['Cycle'] = df_indexed['Cycle'].fillna(-1)
                    df_indexed['Cycle'] = df_indexed['Cycle'].astype(int)
                    df_indexed['Cycle'] = df_indexed['Cycle'].astype(str)
                    df_indexed['Cycle'] = df_indexed['Cycle'].replace('-1', '')
                    df_indexed['Fluorochrome'] = df_indexed['Fluorochrome'].fillna('')

                with tifffile.TiffFile(core_name) as tif:
                    for page in tif.pages:
                        channel_name = df_indexed['Channel_name'][i]
                        if df_indexed.index[i] % 4 == 0:
                            cmap = 'blue'
                        elif df_indexed['Channel_name'][i] == 'Ecadherin':
                            cmap = 'magenta'
                        elif df_indexed['Channel_name'][i] == 'CD4':
                            cmap = 'cyan'
                        elif df_indexed['Channel_name'][i] == 'CD8a':
                            cmap = 'red'
                        elif df_indexed['Channel_name'][i] == 'Vimentin':
                            cmap = 'yellow'
                        else:
                            cmap = 'gray'

                        for tag in page.tags:
                            tag_name, tag_value = tag.name, tag.value

                        image = page.asarray()
                            
                        if 'Cycle' in df.columns and 'Fluorochrome' in df.columns:
                            if df_indexed['Fluorochrome'][i] == '' :
                                fullname = df_indexed['Cycle'][i] + ' ' + channel_name
                            else:
                                fullname = df_indexed['Cycle'][i] + ' ' + channel_name + ' (' + str(df_indexed['Fluorochrome'][i]) + ')'
                        else:
                            fullname = channel_name
                                
                        viewer.add_image(image, name = fullname, colormap=cmap,contrast_limits=(0,65535), visible=False, blending='additive')
                        i = i + 1

    return core_name, channel_name_table, df_name

viewer.window.add_dock_widget(corepicker)

Traceback (most recent call last):
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/psygnal/_signal.py", line 851, in _run_emit_loop
    if caller(args):
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/psygnal/_signal.py", line 1184, in __call__
    self._slot(*args)
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/magicgui/widgets/_function_gui.py", line 212, in _disable_button_and_call
    self.__call__()
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/magicgui/widgets/_function_gui.py", line 324, in __call__
    value = self._function(*bound.args, **bound.kwargs)
  File "/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_6692/2696281695.py", line 33, in corepicker
    if df_indexed.index[i] % 4 == 0:
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 5363, in __getitem__
  

Traceback (most recent call last):
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/psygnal/_signal.py", line 851, in _run_emit_loop
    if caller(args):
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/psygnal/_signal.py", line 1184, in __call__
    self._slot(*args)
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/magicgui/widgets/_function_gui.py", line 212, in _disable_button_and_call
    self.__call__()
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/magicgui/widgets/_function_gui.py", line 324, in __call__
    value = self._function(*bound.args, **bound.kwargs)
  File "/var/folders/02/vkyq99757hsfjng_p8wh1wkc0bfbtx/T/ipykernel_6692/2696281695.py", line 33, in corepicker
    if df_indexed.index[i] % 4 == 0:
  File "/Users/jssarkki/opt/anaconda3/envs/napari_env/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 5363, in __getitem__
  

In [10]:
@magicgui(call_button='Cut Mask')
def cut_mask():
    mask_to_cut = viewer.active_layer.data
    newmask = np.copy(mask_to_cut)
    shape = viewer.active_layer.data.shape
    selected_area = viewer.layers['Shapes'].to_labels(labels_shape=shape)
    global removable_cells
    removable_cells =[]
    
    for i in range(0,mask_to_cut.shape[0]):
        for j in range(0,mask_to_cut.shape[1]):
            cell = mask_to_cut[i,j]
            if selected_area[i,j] > 0 and cell not in removable_cells and cell > 0:
                removable_cells.append(cell)
                print(cell)
    
    celldict = {i: removable_cells[i] for i in range(0,len(removable_cells))}
    
    for i,j in celldict.items():
        newmask[newmask == j] = 0
        
    viewer.add_labels(newmask, name= viewer.active_layer.name + '_cutted')
    
    return(removable_cells)
        
@magicgui(call_button='Save removable cells to csv', filepath={"mode": "w", "mode": "d"})  
def save_list(filepath: Path):
    dict = {'cellid': removable_cells}
    df = pd.DataFrame(dict)
    df = df.astype(int)
    df.to_csv(str(filepath) + '/' + viewer.active_layer.name + '_cellids.csv', index=False)
    
#gui = cut_mask.Gui(show=True)
viewer.window.add_dock_widget(cut_mask)
#gui2 = save_list.Gui(show=True)
viewer.window.add_dock_widget(save_list)


Traceback (most recent call last):
  File "/opt/anaconda3/envs/napari_env/lib/python3.9/site-packages/magicgui/widgets/_bases/value_widget.py", line 57, in _on_value_change
    self.changed.emit(value)
  File "psygnal/_signal.py", line 681, in psygnal._signal.SignalInstance.emit
  File "/opt/anaconda3/envs/napari_env/lib/python3.9/site-packages/magicgui/events.py", line 86, in _run_emit_loop
    cb(*args[:max_args])
  File "/opt/anaconda3/envs/napari_env/lib/python3.9/site-packages/magicgui/widgets/_function_gui.py", line 200, in _disable_button_and_call
    self.__call__()
  File "/opt/anaconda3/envs/napari_env/lib/python3.9/site-packages/magicgui/widgets/_function_gui.py", line 306, in __call__
    value = self._function(*bound.args, **bound.kwargs)
  File "/var/folders/x8/xqkq9cx51dzdw2krk79b8yym0000gn/T/ipykernel_29437/2443885040.py", line 47, in maskpicker
    return img, df_core_cellid, mask_name
NameError: name 'df_core_cellid' is not defined
Traceback (most recent call last):
 

2022-06-09 11:33:49.458 python[29437:580914] -[NSNull length]: unrecognized selector sent to instance 0x7ff8514f50b0
2022-06-09 11:33:49.490 python[29437:580914] Exception calling directoryDidChange: on delegate <QNSOpenSavePanelDelegate: 0x6000004d15e0> exception=-[NSNull length]: unrecognized selector sent to instance 0x7ff8514f50b0
2022-06-09 11:33:49.491 python[29437:580914] -[NSNull length]: unrecognized selector sent to instance 0x7ff8514f50b0
2022-06-09 11:33:49.491 python[29437:580914] Exception calling directoryDidChange: on delegate <QNSOpenSavePanelDelegate: 0x6000004d15e0> exception=-[NSNull length]: unrecognized selector sent to instance 0x7ff8514f50b0
Traceback (most recent call last):
  File "/opt/anaconda3/envs/napari_env/lib/python3.9/site-packages/magicgui/widgets/_bases/value_widget.py", line 57, in _on_value_change
    self.changed.emit(value)
  File "psygnal/_signal.py", line 681, in psygnal._signal.SignalInstance.emit
  File "/opt/anaconda3/envs/napari_env/lib/pyt